# Create an Index Notebook for all of the Notebooks in a folder

This will create an Index Notebook for all of the notebooks in a folder. It will provide a block per notebook which when run will copy the notebook to a working folder, rename it with the date and user appended and open it ready for use.

Perfect for a set of run books.

It requires the ADSNotebook PowerShell module. Run this cell to see if it is available, if not it will install it for your user


In [2]:
#Check for modules and install
$Modules = 'ADSNotebook'


if ((Get-PsRepository -Name PSGallery).InstallationPolicy -ne 'Trusted') {
    Write-Output "The PowerShell Gallery is not trusted so I will trust it so that we can install the modules without interaction"
    try {
        Set-PsRepository -Name PSGallery -InstallationPolicy Trusted
    }
    catch {
        Write-Output " Failed to trust the gallery, trying to force it and also add package provider"
        Install-PackageProvider NuGet -Force
        Import-PackageProvider NuGet -Force
        Set-PSRepository -Name PSGallery -InstallationPolicy Trusted
    }
    
}
else {
    Write-Output "The PowerShell Gallery is trusted I will continue"
}
$Modules.ForEach{
    If (-not(Get-Module $psitem -ListAvailable)) {
        Write-Output "We don't have the $psitem module so we will install it"
        Install-Module $psitem -Scope CurrentUser -Force
    }
    else {
        Write-Output "We have the $psitem module already"
    }
}

We don't have the ADSNotebook module so we will install it


To use this Notebook, you wil need to alter some of the variables  

`$NotebookDirectory` This is the directory holding the notebooks   
`$targetfilename`  This is the name of the Index Notebook  
`$outputLocation` This is the directory that the generated notebook will be placed  


In [2]:
$NotebookDirectory = "C:\Users\rob.sewell\Documents\Work\daa-env-admin\DAA-Operations\01-Notebooks\SSIS"
$targetfilename = "01 - SSIS Notebooks Index.ipynb" 
$outputLocation = 'C:\temp\'
$NotebookPath = "$($NotebookDirectory)\$targetfilename"
$ThisNotebook = 'Create Index Notebook.ipynb'

$IntroCellText = "# Notebook Index for SSIS queries

This Notebook contains the details about the notebooks for each of the SSIS queries 

You can use these Notebooks to investigate an instance  
  
Click on the code below the workbook you require to generate your custom version to save results in"
$IntroCell = New-ADSWorkBookCell -Text $IntroCellText -Type Text

#check output path exists
Write-Output "Starting"
Write-Output "checking Directory $($outputLocation) exists"
If (!(Test-path $outputLocation)) {
    Write-Output "creating Directory $($outputLocation)"
    New-Item -Path  $outputLocation  -ItemType Directory   
} else {
    Write-Output "Directory $($outputLocation) exists"
}

#check ths files available and build the index
Write-Output "Looping through files in  $($NotebookDirectory) and creating the cells"
# add 'pol.TDWH_RiskRetention_S01','pol.TDWH_RiskRetention_S01_copy' to get files in lower directories
$WorkBookCells = foreach ($file in (Get-ChildItem $NotebookDirectory -Exclude $targetfilename,$ThisNotebook)) {
 ## For this one we dont want the sub folders
    # In case there are subfolders which don't have notebooks in them
    If ($file.PSIsContainer -ieq $true -and ((Get-ChildItem $file.FullName -Recurse -Include *ipynb).Count -ne 0)) {                            
  <#      New-ADSWorkBookCell -Type Text -Text "---
---
## <u>Notebook Section:  **$($file.basename) </u>**"   
#>
    }
    Elseif ($file.Extension -eq '.ipynb') {
        $NotebookContent = Get-Content $file.FullName |ConvertFrom-Json
$FirstCell = $NotebookContent.Cells[0].Source -replace '#','##'|Out-String
$Text = "---
---
## Notebook Name: **$($file.basename)**
$FirstCell"
New-ADSWorkBookCell -Type Text -Text $Text       
        $CellText = "# To run a copy of the Notebook $($file.basename), click the run button.
`$user = `$env:USERNAME.Replace('.','')
`$time = get-date -Format 'yyyyMMddHHmmssms'
`$destination = '{3}\{0}_' + `$time + '_'+ `$user + '{1}'
copy-item -path '{2}' -destination `$destination
# In case there is media or script files as well. Copy Those also
Get-ChildItem '{4}' -Exclude *ipynb | Copy-Item -Destination '{3}' -Recurse -Force
azuredatastudio.cmd `$destination
" -f $file.BaseName, $file.Extension, $file.FullName, $outputLocation , $file.DirectoryName
        New-ADSWorkBookCell -Type code -Text $celltext -Collapse
    }
}
Write-Output "Creating the notebook at $($NotebookPath)"
New-ADSWorkBook -path $NotebookPath -Cells $IntroCell, $WorkBookCells -Type PowerShell
Write-Output "Completed"

Starting
checking Directory C:\temp\ exists
Directory C:\temp\ exists
Looping through files in  C:\Users\rob.sewell\Documents\Work\daa-env-admin\DAA-Operations\01-Notebooks\SSIS and creating the cells
Creating the notebook at C:\Users\rob.sewell\Documents\Work\daa-env-admin\DAA-Operations\01-Notebooks\SSIS\01 - SSIS Notebooks Index.ipynb
Completed


In [2]:
pwd


Path          
----          
C:\Users\mrrob


